In [ ]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd

In [ ]:
# ----------------------------
# Set Seed
# ----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# ----------------------------
# EfficientNet-V2-S Deepened
# ----------------------------
class EfficientNetV2S_Deepened_Unfrozen(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.efficientnet_v2_s(weights='IMAGENET1K_V1')

        # Unfreeze all base layers
        for param in self.base_model.parameters():
            param.requires_grad = True


        self.extra_layers = nn.Sequential(
            nn.Conv2d(1280, 1280, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(1280, 1280, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # Replace classifier
        self.classifier = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.extra_layers(x)
        x = self.base_model.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
# ----------------------------
# Dataset & DataLoader
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 41.0MB/s]


In [ ]:
# ----------------------------
# Training & Validation
# ----------------------------
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def validate(model, val_loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / total, accuracy


In [ ]:
# ----------------------------
# Main Training Loop
# ----------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 10
results = []

output_dir = "./emissions_efficientnet_unfrozen"
os.makedirs(output_dir, exist_ok=True)

for seed in [0, 1, 2, 3, 4]:
    print(f"\n🌱 Running Seed {seed}")
    set_seed(seed)

    model = EfficientNetV2S_Deepened_Unfrozen(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # FLOPs Calculation
    macs, params = get_model_complexity_info(model, (3, 32, 32),
                                             as_strings=True,
                                             print_per_layer_stat=False,
                                             verbose=False)
    macs_value = macs.replace('Mac', '').strip()
    if 'G' in macs_value:
        macs_num = float(macs_value.replace('G', '')) * 2
        flops_per_forward = f"{macs_num:.2f} GFLOPs"
    elif 'M' in macs_value:
        macs_num = float(macs_value.replace('M', '')) / 1000 * 2
        flops_per_forward = f"{macs_num:.3f} GFLOPs"
    else:
        flops_per_forward = f"{macs_value} MACs"
    print(f"Seed {seed} → FLOPs per forward pass: {flops_per_forward} | Params: {params}")

    # Energy & CO₂ Tracking
    tracker = EmissionsTracker(
        project_name=f"EfficientNet_Unfrozen_CIFAR10_Seed{seed}",
        output_dir=output_dir,
        measure_power_secs=1
    )
    tracker.start()
    start_time = time.time()

    # Training Loop (4 epochs)
    for epoch in range(4):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Seed {seed} | Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    emissions = tracker.stop()
    elapsed = time.time() - start_time

    results.append((seed, emissions, elapsed, flops_per_forward))

    print(f"🌍 Seed {seed} → CO₂: {emissions:.6f} kg | Time: {elapsed/60:.2f} min | FLOPs: {flops_per_forward}")



🌱 Running Seed 0
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


100%|██████████| 82.7M/82.7M [00:00<00:00, 136MB/s]
[codecarbon WARNING @ 13:39:49] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 13:39:49] [setup] RAM Tracking...
[codecarbon INFO @ 13:39:49] [setup] CPU Tracking...


Seed 0 → FLOPs per forward pass: 0.184 GFLOPs | Params: 50.97 M


[codecarbon WARNING @ 13:39:50] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 13:39:50] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 13:39:50] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 13:39:50] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:39:50] [setup] GPU Tracking...
[codecarbon INFO @ 13:39:50] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:39:50] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 13:39:50] >>> Tracker's metadata:
[codecarbon INFO @ 13:39:50]   Platform sys

Seed 0 | Epoch 1/4 | Train Loss: 1.3924 | Val Acc: 68.68%


...

Seed 4 | Epoch 2/4 | Train Loss: 0.7274 | Val Acc: 78.43%


[codecarbon INFO @ 14:29:30] Energy consumed for RAM : 0.000906 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:29:30] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:29:30] Energy consumed for All CPU : 0.003857 kWh
[codecarbon INFO @ 14:29:30] Energy consumed for all GPUs : 0.006192 kWh. Total GPU Power : 68.89276441684419 W
[codecarbon INFO @ 14:29:30] 0.010955 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:29:31] Energy consumed for RAM : 0.000909 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:29:31] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:29:31] Energy consumed for All CPU : 0.003869 kWh
[codecarbon INFO @ 14:29:31] Energy consumed for all GPUs : 0.006211 kWh. Total GPU Power : 69.53717549802852 W
[codecarbon INFO @ 14:29:31] 0.010989 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:29:32] Energy consumed for RAM : 0.000912 kWh. RAM Power : 1

Seed 4 | Epoch 3/4 | Train Loss: 0.5246 | Val Acc: 81.20%


[codecarbon INFO @ 14:32:16] Energy consumed for RAM : 0.001360 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:32:16] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:32:16] Energy consumed for All CPU : 0.005790 kWh
[codecarbon INFO @ 14:32:16] Energy consumed for all GPUs : 0.009296 kWh. Total GPU Power : 69.78730205326926 W
[codecarbon INFO @ 14:32:16] 0.016447 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:32:17] Energy consumed for RAM : 0.001363 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 14:32:17] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 14:32:17] Energy consumed for All CPU : 0.005802 kWh
[codecarbon INFO @ 14:32:17] Energy consumed for all GPUs : 0.009314 kWh. Total GPU Power : 63.09204116420289 W
[codecarbon INFO @ 14:32:17] 0.016479 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:32:18] Energy consumed for RAM : 0.001366 kWh. RAM Power : 1

Seed 4 | Epoch 4/4 | Train Loss: 0.3802 | Val Acc: 82.53%
🌍 Seed 4 → CO₂: 0.007643 kg | Time: 11.03 min | FLOPs: 0.184 GFLOPs


In [ ]:
# ----------------------------
# Results Summary
# ----------------------------
results_df = pd.DataFrame(results, columns=['seed', 'co2_kg', 'time_s', 'flops'])
print("\n--- Summary Statistics ---")
print(f"CO₂ Emissions: Mean = {results_df['co2_kg'].mean():.6f} kg, Std Dev = {results_df['co2_kg'].std():.6f} kg")
print(f"Training Time: Mean = {results_df['time_s'].mean()/60:.2f} min, Std Dev = {results_df['time_s'].std()/60:.2f} min")


--- Summary Statistics ---
CO₂ Emissions: Mean = 0.007621 kg, Std Dev = 0.000035 kg
Training Time: Mean = 11.00 min, Std Dev = 0.04 min


In [ ]:
import pandas as pd
import os

output_dir = "./emissions_efficientnet_unfrozen"  # same as used during training
csv_file = os.path.join(output_dir, "emissions.csv")

# Read the CodeCarbon CSV
df = pd.read_csv(csv_file)

# Prepare results with energy
results_with_energy = []

for seed, emissions, elapsed, flops in results:
    project_name = f"EfficientNet_Unfrozen_CIFAR10_Seed{seed}"

    # Fetch the last entry for that seed
    row = df[df['project_name'] == project_name].iloc[-1]
    energy = row['energy_consumed']  # kWh

    results_with_energy.append((seed, emissions, elapsed, flops, energy))

# Print results summary including energy
print("\n--- Results Summary ---")
for seed, emissions, elapsed, flops, energy in results_with_energy:
    print(f"Seed {seed}: CO₂ = {emissions:.6f} kg, Time = {elapsed/60:.2f} min, FLOPs = {flops}, Energy = {energy:.6f} kWh")



--- Results Summary ---
Seed 0: CO₂ = 0.007559 kg, Time = 10.93 min, FLOPs = 0.184 GFLOPs, Energy = 0.021644 kWh
Seed 1: CO₂ = 0.007639 kg, Time = 11.02 min, FLOPs = 0.184 GFLOPs, Energy = 0.021875 kWh
Seed 2: CO₂ = 0.007628 kg, Time = 11.01 min, FLOPs = 0.184 GFLOPs, Energy = 0.021842 kWh
Seed 3: CO₂ = 0.007637 kg, Time = 11.02 min, FLOPs = 0.184 GFLOPs, Energy = 0.021869 kWh
Seed 4: CO₂ = 0.007643 kg, Time = 11.03 min, FLOPs = 0.184 GFLOPs, Energy = 0.021886 kWh


In [ ]:
import numpy as np

# Extract columns
co2 = np.array([r[1] for r in results_with_energy])
time_min = np.array([r[2]/60 for r in results_with_energy])  # convert seconds to minutes
energy = np.array([r[4] for r in results_with_energy])

# Compute mean and standard deviation
co2_mean, co2_std = co2.mean(), co2.std()
time_mean, time_std = time_min.mean(), time_min.std()
energy_mean, energy_std = energy.mean(), energy.std()

print("\n--- Summary Statistics ---")
print(f"CO₂ Emissions: Mean = {co2_mean:.6f} kg, Std Dev = {co2_std:.6f} kg")
print(f"Training Time: Mean = {time_mean:.2f} min, Std Dev = {time_std:.2f} min")
print(f"Energy Consumption: Mean = {energy_mean:.6f} kWh, Std Dev = {energy_std:.6f} kWh")



--- Summary Statistics ---
CO₂ Emissions: Mean = 0.007621 kg, Std Dev = 0.000032 kg
Training Time: Mean = 11.00 min, Std Dev = 0.04 min
Energy Consumption: Mean = 0.021823 kWh, Std Dev = 0.000091 kWh
